# Next‑Level NumPy (Post‑Basics)

This notebook builds on fundamentals and focuses on the mental models that make NumPy fast, correct, and expressive.


In [6]:
import numpy as np

np.set_printoptions(precision=4, suppress=True)


## 1. Shape Reasoning & Broadcasting (Top Priority)
Broadcasting rules (right‑alignment, size‑1 expansion) and explicit dimension insertion.


In [7]:
# Right‑alignment and size‑1 expansion
A = np.arange(6).reshape(2, 3)   # shape (2,3)
print(A, end="\n\n")
b = np.array([10, 20, 30])       # shape (3,)
print(b)

A + b   # b is treated as (1,3) then expanded to (2,3)

[[0 1 2]
 [3 4 5]]

[10 20 30]


array([[10, 21, 32],
       [13, 24, 35]])

In [10]:
# Explicit dimension insertion
x = np.array([1, 2, 3])          # (3,)

print(x[:, None].shape)                 # (3,1)
print(x[None, :].shape)                 # (1,3)


(3, 1)
(1, 3)


In [18]:
# Predict output shape mentally
X = np.random.randn(5, 3)        # (n=5, d=3)

print((X - X.mean(axis = 0)) / X.std(axis = 0))     # convert to z-scores


[[ 0.7223  0.4464 -0.2498]
 [ 1.3009  1.7636  0.0329]
 [ 0.1582 -0.8043  1.1676]
 [-1.5277 -0.5363 -1.7246]
 [-0.6537 -0.8694  0.774 ]]


In [ ]:
# Pairwise ops without loops
X = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])  # (3,2)

pairwise_diff = X[:, None, :] - X[None, :, :]      # (3,3,2)
pairwise_diff

array([[[ 0.,  0.],
        [-2., -2.],
        [-4., -4.]],

       [[ 2.,  2.],
        [ 0.,  0.],
        [-2., -2.]],

       [[ 4.,  4.],
        [ 2.,  2.],
        [ 0.,  0.]]])

In [27]:
# exercise part 1
squared_diff = np.square(X[:, None, :] - X[None, :, :])
squared_diff

array([[[ 0.,  0.],
        [ 4.,  4.],
        [16., 16.]],

       [[ 4.,  4.],
        [ 0.,  0.],
        [ 4.,  4.]],

       [[16., 16.],
        [ 4.,  4.],
        [ 0.,  0.]]])

In [35]:
# exercise part 2
manhattan = np.abs(X[:, None, :] - X[None, :, :]).sum(axis=2)
manhattan

array([[0., 4., 8.],
       [4., 0., 4.],
       [8., 4., 0.]])

## 2. Axis Semantics (Beyond “axis=0 means columns”)
Reduction vs transformation, `keepdims=True`, and behavior under reshape/transpose.


In [ ]:
X = np.arange(1, 13).reshape(4, 3)
X


In [ ]:
# Reduction along axis
X.sum(axis=0)   # per‑feature (column) sum
X.sum(axis=1)   # per‑sample (row) sum


In [ ]:
# keepdims preserves dimensionality for broadcasting
mean0 = X.mean(axis=0, keepdims=True)  # (1,3)
X_centered = X - mean0                 # broadcasts correctly
X_centered


In [ ]:
# Row‑wise normalization
row_norms = np.linalg.norm(X, axis=1, keepdims=True)
X_normalized = X / row_norms
X_normalized


In [ ]:
# Axis behavior under transpose
Xt = X.T
X.shape, Xt.shape


## 3. Views, Copies, and Fancy Indexing Traps
Slicing usually returns a **view**; fancy indexing always returns a **copy**.


In [ ]:
arr = np.arange(10)
view = arr[2:7]       # slice view
copy = arr[[2, 3, 4]] # fancy index copy

np.shares_memory(arr, view), np.shares_memory(arr, copy)


In [ ]:
view[:] = 99
arr


In [ ]:
copy[:] = 77
arr  # unchanged by copy edits


In [ ]:
# base attribute points to original owning array (or None)
view.base is arr, copy.base


## 4. Memory Layout & Strides (Mental Model)
Contiguity and strides explain why some ops are free and others allocate.


In [ ]:
X = np.arange(12).reshape(3, 4)
X.flags['C_CONTIGUOUS'], X.flags['F_CONTIGUOUS'], X.strides


In [ ]:
Xt = X.T
Xt.flags['C_CONTIGUOUS'], Xt.flags['F_CONTIGUOUS'], Xt.strides


In [ ]:
# Reshape can be free if memory is compatible
Y = X.reshape(2, 6)
Y


In [ ]:
# Transpose often returns a view with different strides
Xt = X.T
Xt


(Optional) `as_strided` can create sliding windows without copying.
Use with care — incorrect strides can crash Python or return invalid data.


In [ ]:
from numpy.lib.stride_tricks import as_strided

x = np.arange(8)
# Create a 5x4 sliding window view over x
window_shape = (5, 4)
strides = (x.strides[0], x.strides[0])
windows = as_strided(x, shape=window_shape, strides=strides)
windows


## 5. Vectorization Patterns (Algorithmic Thinking)
Broadcast + reduce, mask + assign, reshape → operate → reshape back.


In [ ]:
# Pairwise distance matrix (squared Euclidean)
X = np.random.randn(4, 3)

# (n,1,d) - (1,n,d) -> (n,n,d), then sum over d
D2 = ((X[:, None, :] - X[None, :, :]) ** 2).sum(axis=2)
D2


In [ ]:
# Mask + assign
x = np.linspace(-2, 2, 9)
y = x.copy()

y[x < 0] = 0

y


In [ ]:
# Reshape -> operate -> reshape back
img = np.arange(3*4*2).reshape(3, 4, 2)  # (H,W,C)
flat = img.reshape(-1, 2)               # (H*W, C)
flat = flat - flat.mean(axis=0, keepdims=True)
img_centered = flat.reshape(3, 4, 2)
img_centered


## 6. Boolean Logic as Control Flow
Compound masks, `np.where`, and data filtering pipelines.


In [ ]:
x = np.linspace(-3, 3, 13)

mask = (x >= -1) & (x <= 1)
filtered = x[mask]

mask, filtered


In [ ]:
# Piecewise function using np.where
f = np.where(x < 0, x**2, np.sqrt(x))
f


In [ ]:
# Clipping and thresholding
np.clip(x, -1, 1)


## 7. Numerical Linear Algebra (Used Correctly)
Avoid explicit inverse; prefer `solve`, `lstsq`, and `pinv`.


In [ ]:
A = np.array([[3.0, 1.0], [1.0, 2.0]])
b = np.array([9.0, 8.0])

x = np.linalg.solve(A, b)
x


In [ ]:
# Least squares (overdetermined)
X = np.array([[1, 1], [1, 2], [1, 3]], dtype=float)
y = np.array([1, 2, 2], dtype=float)

beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
beta, residuals, rank


In [ ]:
# Rank deficiency and pseudoinverse
A = np.array([[1, 2], [2, 4]], dtype=float)  # rank 1
np.linalg.matrix_rank(A), np.linalg.pinv(A)


In [ ]:
# PCA without libraries via SVD
X = np.random.randn(100, 3)
X = X - X.mean(axis=0, keepdims=True)

U, S, Vt = np.linalg.svd(X, full_matrices=False)
components = Vt   # principal axes
explained_variance = (S**2) / (len(X) - 1)
components, explained_variance


## 8. Numerical Stability & Dtypes
Floating‑point error, overflow/underflow, stable formulations.


In [ ]:
# float32 vs float64
x32 = np.array([1e10, 1.0], dtype=np.float32)
x64 = np.array([1e10, 1.0], dtype=np.float64)

x32 - 1e10, x64 - 1e10


In [ ]:
# Stable softmax
z = np.array([1000.0, 1001.0, 1002.0])

exp_naive = np.exp(z) / np.exp(z).sum()
exp_stable = np.exp(z - z.max()) / np.exp(z - z.max()).sum()

exp_naive, exp_stable


In [ ]:
# Comparing floats safely
a = 0.1 + 0.2
b = 0.3
np.isclose(a, b)


## 9. Randomness & Simulation (Modern NumPy)
Use `default_rng` for reproducibility and performance.


In [ ]:
rng = np.random.default_rng(123)

rng.random(5)


In [ ]:
# Monte Carlo estimate of pi
rng = np.random.default_rng(0)
N = 100_000
pts = rng.random((N, 2))
inside = (pts**2).sum(axis=1) <= 1.0
pi_est = 4 * inside.mean()
pi_est


In [ ]:
# Random walk (vectorized)
steps = rng.choice([-1, 1], size=1000)
walk = steps.cumsum()
walk[:10]


## 10. `einsum` (Optional but Elite)
Express complex operations compactly.


In [ ]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

np.einsum('ij,jk->ik', A, B)  # matrix multiplication


In [ ]:
# Trace and Frobenius norm
np.einsum('ii', A)            # trace
np.sqrt(np.einsum('ij,ij', A, A))


## 11. Performance Awareness
Avoid unnecessary temporaries, check contiguity, and time critical paths.


In [ ]:
import timeit

X = np.random.randn(1000, 1000)

# Two ways to compute row-wise mean centering

def center_naive():
    return X - X.mean(axis=0)

def center_keepdims():
    return X - X.mean(axis=0, keepdims=True)

# keepdims avoids reshaping or broadcasting mistakes, same speed here
naive_time = timeit.timeit(center_naive, number=5)
keepdims_time = timeit.timeit(center_keepdims, number=5)

naive_time, keepdims_time


In [ ]:
# Contiguity checks
X.flags['C_CONTIGUOUS'], X.T.flags['C_CONTIGUOUS']
